# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [74]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import pandas as pd

#### Creating list of filepaths to process original event csv data files

In [72]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [61]:
full_data_rows_list = [] 
    
# Iterate through every filepath in the file path list, extract data rows & append to the full list:
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Creating a smaller event data csv file that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [76]:
# Check number of data rows and columns in the file

df = pd.read_csv('event_datafile_new.csv')

df.shape

(6820, 11)

In [64]:
# Check check columns and their order
for i, c in enumerate(df.columns):
    print(i, c)

0 artist
1 firstName
2 gender
3 itemInSession
4 lastName
5 length
6 level
7 location
8 sessionId
9 song
10 userId


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Initiate a Cassandra instance

#### Creating a Cluster

In [65]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [66]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = { 'class': 'SimpleStrategy', 'replication_factor': 1 }    
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [67]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


## Create tables and  queries to answer the following questions:

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

For this question, we create a table called `events_session` with the following schema:
- Fields: artist, song, length, sessionId, itemInSession
- Primary key: (sessionId, ItemInSession) because the query will filter on these two fields


In [77]:
create_table_events_session = """
    CREATE TABLE IF NOT EXISTS events_session (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length float,
        PRIMARY KEY (sessionId, itemInSession)
    )
"""

try:
    session.execute(create_table_events_session)
except Exception as e:
    print(e)

#### Read CSV file and insert into the `events_session` table:

In [78]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO events_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(
            query, 
            (
                int(line[8]) 
                , int(line[3]) 
                , line[0] 
                , line[9] 
                , float(line[5]) 
                
            )
        )

#### Do a SELECT to verify that the data have been inserted

In [79]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query_1 = """
    select 
        artist, 
        song, 
        length
    from events_session
    where sessionId = 338 
        and itemInSession = 4
"""

try:
    rows = session.execute(query_1)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For this question, we will create an `events_user_session` table with the following schema:
- Fields: artist, song, firstName, LastName, userId, sessionId, itemInSession
- Primary key: ((userId, sessionId), itemInSession). 

We need the primary key to be unique, and in this case the combination of `userId` - `sessionId` - `itemInSession` satisfies this condition.

The songs need to be sorted by `itemInSession` so this will be the clustering key. We will also filter on the userID and session ID, so this combination will be the partition key.

In [82]:
create_table_events_user_session = """ 
    CREATE TABLE IF NOT EXISTS events_user_session (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        
        PRIMARY KEY ((userId, sessionId), itemInSession)
    )
"""

try:
    session.execute(create_table_events_user_session)
except Exception as e:
    print(e)


#### Read CSV file and insert into the `events_user_session` table:

In [83]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    for line in csvreader:
        query = "INSERT INTO events_user_session (userId, sessionId, itemInsession, song, artist, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(
            query, 
            (
                int(line[10])
                , int(line[8]) 
                , int(line[3]) 
                , line[9] 
                , line[0]
                , line[1] 
                , line[4] 
            )
        )

#### Do a SELECT to verify that the data have been inserted

In [84]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query_2 = """
    select 
        artist, 
        song,
        firstName,
        lastName
    from events_user_session
    where sessionId = 182
        and userId = 10 
"""

                    

try:
    rows = session.execute(query_2)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

For this question, we will create an `events_song` table with the following schema:
- Fields: userId, firstName, lastName, song
- Primary Key: (song, userId). 

`song` needs to be the partition key because we will filter against the song name. We also need a unique combination that includes `song`, and the **song + user** combination makes a good unique identifier here.

In [85]:
create_table_events_song = """
    CREATE TABLE IF NOT EXISTS events_song (
        song text,
        userId int,
        firstName text,
        lastName text,
        
        PRIMARY KEY (song, userId)
    )
"""

try:
    session.execute(create_table_events_song)
except Exception as e:
    print(e)


#### Read CSV file and insert into the `events_user_session` table:

In [86]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO events_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        session.execute(
            query, 
            (
                line[9] 
                , int(line[10])
                , line[1] 
                , line[4] 
            )
        )

#### Do a SELECT to verify that the data have been inserted

In [87]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query_3 = """
    select 
        firstName, 
        lastName
    from events_song
    where song = 'All Hands Against His Own'
"""

                    

try:
    rows = session.execute(query_3)
    for row in rows:
        print(row)
except Exception as e:
    print(e)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [36]:
## TO-DO: Drop the table before closing out the sessions

session.execute("DROP TABLE events_session")
session.execute("DROP TABLE events_user_session")
session.execute("DROP TABLE events_song")


### Close the session and cluster connection¶

In [37]:
session.shutdown()
cluster.shutdown()